In [1]:
import time
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
import warnings

In [2]:
def mse(predicted,real):
    return ((real-predicted)**2).sum()/len(real)

def first_method(x,y):
    start=time.time()
    x[x<0.1]=0.1 #log(0) -infinite overflow prevention
    X = np.log(x).reshape(-1,1)
    Y = np.log(y).reshape(-1,1)
    reg = LinearRegression().fit(X, Y)
    predictions=reg.predict(X)
    end=time.time()
    mse1=mse(np.squeeze(np.exp(predictions.reshape(1,-1))),y)
    return mse1,(end-start)
#********************************************************************************************************  
def func(x, a, b, c):
    return a * np.exp(-b * x) + c    
    
def third_method(x,y):
    start=time.time()
    popt, pcov = curve_fit(func, x, y)
    end=time.time()
    mse3=mse(func(x, *popt),y)
    return mse3,(end-start)

In [3]:
import pandas as pd
df=pd.read_csv("New_volume_forecast_with_sectorial_variational_appending_by_Ulas.csv",sep=";")
df

,FCOPERID,datetime,CELL,forecasteddlprbutilbh,actualdlprbutilbh,forecastedtputbh,actualdltputbh
0,E302BBF16BA481458B54787E601EFC25,2019-10-02,KRKTP17L,NaN,NaN,NaN,NaN
1,E302BBF16BA481458B54787E601EFC25,2019-10-03,KRKTP17L,NaN,NaN,NaN,NaN
2,E302BBF16BA481458B54787E601EFC25,2019-10-04,KRKTP17L,NaN,NaN,NaN,NaN
3,E302BBF16BA481458B54787E601EFC25,2019-10-05,KRKTP17L,NaN,NaN,NaN,NaN
4,E302BBF16BA481458B54787E601EFC25,2019-10-06,KRKTP17L,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
4918843,E302BBF16BA481458B54787E601EFC25,2021-12-27,ZAKUM37L,29.222239,71.52,24.899002,13.166
4918844,E302BBF16BA481458B54787E601EFC25,2021-12-28,ZAKUM37L,29.222239,69.51,24.899002,15.281
4918845,E302BBF16BA481458B54787E601EFC25,2021-12-29,ZAKUM37L,29.222239,70.03,24.899002,10.509
4918846,E302BBF16BA481458B54787E601EFC25,2021-12-30,ZAKUM37L,29.222239,75.93,24.899002,13.901


In [4]:
df=df.replace([np.inf, -np.inf], np.nan)
df=df.dropna(subset=['actualdlprbutilbh',"CELL","actualdltputbh"], how="any")
df=df[["CELL","actualdlprbutilbh","actualdltputbh"]].drop_duplicates()
df.describe()

,actualdlprbutilbh,actualdltputbh
count,2.090488e+06,2.090488e+06
mean,3.956716e+01,1.999538e+01
std,2.538467e+01,1.514766e+01
min,3.400000e-01,-1.934000e+00
25%,1.831000e+01,9.754000e+00
50%,3.582000e+01,1.660400e+01
75%,5.810000e+01,2.606900e+01
max,9.998000e+01,1.293000e+03


In [8]:
warnings.filterwarnings("ignore")

results_df=pd.DataFrame(columns=["Method Name","Total MSE","Average MSE","Total Time Passed"])
results1_df=pd.DataFrame(columns=["Cell","MSE","Passed"])
results3_df=pd.DataFrame(columns=["Cell","MSE","Passed"])


total_mse_1,total_mse_3=0,0
total_time_passed_1,total_time_passed_3=0,0
counter=0

for cell in set(df.CELL.values):
    y=(df[cell==df.CELL].loc[:,"actualdltputbh"]).to_numpy()
    x=(df[cell==df.CELL].loc[:,"actualdlprbutilbh"]).to_numpy()
    
    try:
        mse1,time_passed1 = first_method(x,y)
        if np.isnan(mse1):
            mse1=-1
        total_mse_1 +=mse1
        total_time_passed_1 +=time_passed1
        results1_df=results1_df.append({"Cell":cell,"MSE":"{:.2f}".format(mse1),
                                        "Passed":"{:.6f}".format(time_passed1)},ignore_index=True)
    except:
        pass
    
    try:
        mse3,time_passed3 = third_method(x,y)
        if np.isnan(mse3):
            mse3=-1
        total_mse_3 += mse3
        total_time_passed_3 +=time_passed3
        results3_df=results3_df.append({"Cell":cell,"MSE":"{:.2f}".format(mse3),
                                        "Passed":"{:.6f}".format(time_passed3)},ignore_index=True)
    except:
        pass
    counter+=1
    print(counter)
    if counter==300:
        break

results_df=results_df.append({"Method Name":"Linear Regression","Average MSE":"{:.2f}".format(total_mse_1/counter),
                              "Total MSE":"{:.2f}".format(total_mse_1),
                              "Total Time Passed":"{:.2f}".format(total_time_passed_1)},ignore_index=True)
results_df=results_df.append({"Method Name":"Curve Fit","Average MSE":"{:.2f}".format(total_mse_3/counter),
                              "Total MSE":"{:.2f}".format(total_mse_3),
                              "Total Time Passed":"{:.2f}".format(total_time_passed_3)},ignore_index=True)
results_df.to_csv(r"results.csv")
results1_df.to_csv(r"results_linear.csv")
results3_df.to_csv(r"results_curve_fit.csv")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
print(1)

In [ ]:
len(set(df.CELL.values))